In [125]:
import pandas as pd

import sql_functions as sf

In [126]:
schema = 'organic_africa' # UPDATE 'TABLE_SCHEMA' based on schema used in class 
engine = sf.get_engine()

In [127]:
pur_tables = {}
pur_tables['OGPur__OGMB'] = sf.get_dataframe(f'SELECT * FROM {schema}."OGPur__OGMB"')
pur_tables['OG_Pur__OGOrg_1'] = sf.get_dataframe(f'SELECT * FROM {schema}."OG_Pur__OGOrg_1"')
pur_tables['wc_pur__WCOrg_1'] = sf.get_dataframe(f'SELECT * FROM {schema}."wc_pur__WCOrg_1"')
pur_tables['wc_pur_deviations__WCOrg_1'] = sf.get_dataframe(f'SELECT * FROM {schema}."wc_pur_deviations__WCOrg_1"')
pur_tables['wc_purchases_bayoba_1'] = sf.get_dataframe(f'SELECT * FROM {schema}."wc_purchases_bayoba_1"')
pur_tables['wc_purchases_deviations_bayoba_1'] = sf.get_dataframe(f'SELECT * FROM {schema}."wc_purchases_deviations_bayoba_1"')

In [128]:
del pur_tables['OGPur__OGMB']['Input_recovery']
del pur_tables['OG_Pur__OGOrg_1']['Input_recovery']

In [129]:
# column names lower case

for df in pur_tables.values():
    #print(df)
    df.columns = map(str.lower, df.columns)

pur_tables['OGPur__OGMB'].columns

Index(['id', 'og_nr', 'name_verified', 'date_of_purchase', 'receipt_nr',
       'product', 'grade', 'amount_in_kg', 'price_per_kg', 'total_payment_usd',
       'buyer', 'agr_year', 'comment', 'entered_by', 'source'],
      dtype='object')

In [130]:
pur_tables['OGPur__OGMB'].loc[pur_tables['OGPur__OGMB']['date_of_purchase']=='037/2021']
# 4097

,id,og_nr,name_verified,date_of_purchase,receipt_nr,product,grade,amount_in_kg,price_per_kg,total_payment_usd,buyer,agr_year,comment,entered_by,source
4097,6657,18493.0,False,037/2021,6091.0,Rosella (subdariffa) petals (MB),A,11.0,0.85,9.0,Philip,2021.0,None,Nyasha,OGOrg


In [131]:
pur_tables['OGPur__OGMB']['date_of_purchase'] = pur_tables['OGPur__OGMB'].iloc[4097]['date_of_purchase'].replace('037/2021', '03/07/2021')
pur_tables['OGPur__OGMB'].iloc[4097]['date_of_purchase']

'03/07/2021'

In [132]:
# Function to pad/fill to short dates like 5/3/18 to 05/03/2018

def str_date_pad(val):
    
    if type(val) is str and len(val) < 10: 

        date_splitted = val.split('/')
        print(date_splitted)
        if len(date_splitted[1]) < 2 : 
            date_splitted[1] = '0' + date_splitted[1]

        if (len(date_splitted[2]) < 4) and (int(date_splitted[2]) < 24) : 
            date_splitted[2] = '20' + date_splitted[2]
        if len(date_splitted[2]) < 4 and int(date_splitted[2]) > 23 : 
            date_splitted[2] = '19' + date_splitted[2]
    
        val = '/'.join(date_splitted)
        #print(val)
        return val
    
    else:
        return val

In [133]:
# Date_of_purchase to_datetime
pur_tables['OGPur__OGMB']['date_of_purchase'] = pur_tables['OGPur__OGMB']['date_of_purchase'].apply(str_date_pad)

In [134]:
pur_tables['OGPur__OGMB']['date_of_purchase'] = pd.to_datetime(pur_tables['OGPur__OGMB']['date_of_purchase'])

In [135]:
pur_tables['OG_Pur__OGOrg_1']['date_of_purchase'] = pd.to_datetime(pur_tables['OG_Pur__OGOrg_1']['date_of_purchase'])
pur_tables['wc_pur__WCOrg_1']['date_of_purchase'] = pd.to_datetime(pur_tables['wc_pur__WCOrg_1']['date_of_purchase'])
pur_tables['wc_pur_deviations__WCOrg_1']['date_of_purchase'] = pd.to_datetime(pur_tables['wc_pur_deviations__WCOrg_1']['date_of_purchase'])
pur_tables['wc_purchases_bayoba_1']['date_of_purchase'] = pd.to_datetime(pur_tables['wc_purchases_bayoba_1']['date_of_purchase'])
pur_tables['wc_purchases_deviations_bayoba_1']['date_of_purchase'] = pd.to_datetime(pur_tables['wc_purchases_deviations_bayoba_1']['date_of_purchase'])

/var/folders/x5/v1d8l9v94679fhr8vd07ys9c0000gn/T/ipykernel_1053/127177431.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  pur_tables['OG_Pur__OGOrg_1']['date_of_purchase'] = pd.to_datetime(pur_tables['OG_Pur__OGOrg_1']['date_of_purchase'])


#### Concat

In [136]:
for name, df in pur_tables.items():
    print(name)
    print(df.shape)

OGPur__OGMB
(6612, 15)
OG_Pur__OGOrg_1
(4144, 15)
wc_pur__WCOrg_1
(2731, 15)
wc_pur_deviations__WCOrg_1
(464, 15)
wc_purchases_bayoba_1
(10451, 15)
wc_purchases_deviations_bayoba_1
(650, 15)


In [137]:
concated_pur = pd.concat(pur_tables)
concated_pur.dtypes

id                            int64
og_nr                       float64
name_verified                  bool
date_of_purchase     datetime64[ns]
receipt_nr                   object
product                      object
grade                        object
amount_in_kg                float64
price_per_kg                float64
total_payment_usd           float64
buyer                        object
agr_year                     object
comment                      object
entered_by                   object
source                       object
wc_nr                       float64
dtype: object

#### Further Cleaning
- Receipt_Nr to_numeric

In [138]:
# concated_pur['receipt_nr']= pd.to_numeric(concated_pur['receipt_nr'])
# Unable to parse string "1753A" at position 8885

concated_pur['receipt_nr'] = concated_pur.iloc[8885]['receipt_nr'].replace('1753A', '1753')

concated_pur.iloc[8885]['receipt_nr']

'1753'

In [139]:
concated_pur['receipt_nr']= pd.to_numeric(concated_pur['receipt_nr'])

In [140]:
type(concated_pur['wc_nr'])

pandas.core.series.Series

In [141]:
wc = concated_pur.loc[pd.notna(concated_pur['wc_nr'])==True,'wc_nr']
for nr in wc: 
    if str(nr).endswith('.0')==False: print(nr)

0.268


In [142]:
0.268
concated_pur['wc_nr'] = concated_pur['wc_nr'].replace('0.268', '268.0')

In [143]:
# Correcting outliers

concated_pur['agr_year'] = concated_pur['agr_year'].replace('2013/14', '2013')
concated_pur['agr_year'] = concated_pur['agr_year'].replace('2014/15', '2014')
concated_pur['agr_year'] = concated_pur['agr_year'].replace('2021q', '2021')

In [144]:
concated_pur['agr_year'] = pd.to_datetime(concated_pur['agr_year'])
#concated_pur['agr_year'] = concated_pur['agr_year'].dt.year

In [145]:
concated_pur['agr_year'] = concated_pur['agr_year'].dt.strftime('%Y')
#concated_pur['agr_year'] = pd.to_datetime(concated_pur['agr_year']).dt.year

In [146]:
display(concated_pur.dtypes)
display(concated_pur.shape)
display(concated_pur.head())

id                            int64
og_nr                       float64
name_verified                  bool
date_of_purchase     datetime64[ns]
receipt_nr                    int64
product                      object
grade                        object
amount_in_kg                float64
price_per_kg                float64
total_payment_usd           float64
buyer                        object
agr_year                     object
comment                      object
entered_by                   object
source                       object
wc_nr                       float64
dtype: object

(25052, 16)

id    og_nr  name_verified date_of_purchase  receipt_nr  \
OGPur__OGMB 0  1576  13232.0          False       2021-03-07        1753   
            1  1577  13237.0          False       2021-03-07        1753   
            2  1578  13241.0          False       2021-03-07        1753   
            3  1579  13105.0          False       2021-03-07        1753   
            4  1580  13032.0          False       2021-03-07        1753   

                                        product grade  amount_in_kg  \
OGPur__OGMB 0  Rosella (subdariffa) petals (MB)     A        15.260   
            1  Rosella (subdariffa) petals (MB)     A        74.555   
            2  Rosella (subdariffa) petals (MB)     A        61.855   
            3  Rosella (subdariffa) petals (MB)     A         6.955   
            4  Rosella (subdariffa) petals (MB)     A         7.885   

               price_per_kg  total_payment_usd   buyer agr_year comment  \
OGPur__OGMB 0          0.85               13.0  Philip     1970    None   
            1          0.85               63.0  Philip     1970    None   
            2          0.85               53.0  Philip     1970    None   
            3          0.85                6.0  Philip     1970    None   
            4          0.85                7.0  Philip     1970    None   

              entered_by source  wc_nr  
OGPur__OGMB 0     Nyasha  OGOrg    NaN  
            1     Nyasha  OGOrg    NaN  
            2     Nyasha  OGOrg    NaN  
            3     Nyasha  OGOrg    NaN  
            4     Nyasha  OGOrg    NaN

##### check duplicates and unique columns

In [147]:
# Looking for Duplicates
concated_pur.duplicated().value_counts()

# NO Duplicates

False    25052
dtype: int64

In [148]:
# sum of null values each column
concated_pur.isnull().sum()

# comment 24300 nulls

id                       0
og_nr                14321
name_verified            0
date_of_purchase       181
receipt_nr               0
product                 14
grade                13825
amount_in_kg             3
price_per_kg            25
total_payment_usd        3
buyer                   16
agr_year                16
comment              24300
entered_by             132
source                   2
wc_nr                11015
dtype: int64

In [149]:
concated_pur[['og_nr']].squeeze().is_unique

False

In [150]:
concated_pur['product'] = concated_pur['product'].replace({ 'Rosella (subdariffa) petals (MB)': 'Rosella (subdariffa) petals (org)',
                                                            'Rosella (subdariffa) seed (MB)': 'Rosella (subdariffa) seed (org)',
                                                            'Devil’s Claw plant part (org) wet': 'Devil’s Claw plant part (org)',
                                                            'kalahari melon seed (org)': 'Kalahari melon seed (org)'})

concated_pur['grade'] = concated_pur['grade'].replace(' ', None)

In [151]:
for col in ['name_verified','receipt_nr', 'entered_by', 'buyer', 'grade','product']:
    display(col)
    display(concated_pur[col].value_counts())

'name_verified'

False    17456
True      7596
Name: name_verified, dtype: int64

'receipt_nr'

1753    25052
Name: receipt_nr, dtype: int64

'entered_by'

...

Daniel             5146
Douglas            4227
Islum              2944
Philip             2404
Charles            2044
Wiseman            1528
Memory             1381
Andrew              900
Misheck             855
Ronald              755
Kennedy             703
Doreen              643
Pedzisai            642
Fungai              374
Hamid               193
Hadley              141
Nyasha              116
Nyasha Poto-Joe      28
Mandeya               9
d                     1
Meki Saizi            1
Katinka               1
Name: buyer, dtype: int64

'grade'

A    11187
B       23
C       15
Name: grade, dtype: int64

'product'

Baobab fruit whole (org)               9231
Rosella (subdariffa) petals (org)      7938
Gotu kola leaves (org)                 2086
Bird's Eye Chilli fruit whole (org)    1173
Rosella (subdariffa) seed (org)        1080
Baobab whole fruit (con)                650
Devil’s Claw root (org)                 519
Paprika fruit whole (org)               384
Devil’s Claw root (con)                 314
Trichillia emetica                      246
Ximenia Americana seed (org)            240
Baobab pulp and seed (org)              216
Strophanthus (kombe) seed (con)         163
Kalahari melon seed (org)               145
Ximenia Caffra Seed (org)               126
Gotu kola leaves (con)                  116
Marula seed (org)                        95
Devil’s Claw plant part (org)            78
Trichillia emetica (org)                 68
Marula Kennels                           41
Strophanthus (kombe) seed (org)          40
Devil’s Claw plant part (con)            32
Rosella petals (con)            

In [152]:
concated_pur

id    og_nr  name_verified  \
OGPur__OGMB                      0    1576  13232.0          False   
                                 1    1577  13237.0          False   
                                 2    1578  13241.0          False   
                                 3    1579  13105.0          False   
                                 4    1580  13032.0          False   
...                                    ...      ...            ...   
wc_purchases_deviations_bayoba_1 645   646      NaN          False   
                                 646   647      NaN          False   
                                 647   648      NaN          False   
                                 648   649      NaN          False   
                                 649   650      NaN          False   

                                     date_of_purchase  receipt_nr  \
OGPur__OGMB                      0         2021-03-07        1753   
                                 1         2021-03-07        1753   
                                 2         2021-03-07        1753   
                                 3         2021-03-07        1753   
                                 4         2021-03-07        1753   
...                                               ...         ...   
wc_purchases_deviations_bayoba_1 645       2015-08-07        1753   
                                 646       2015-08-04        1753   
                                 647       2015-08-31        1753   
                                 648       2015-08-31        1753   
                                 649       2015-09-10        1753   

                                                                product grade  \
OGPur__OGMB                      0    Rosella (subdariffa) petals (org)     A   
                                 1    Rosella (subdariffa) petals (org)     A   
                                 2    Rosella (subdariffa) petals (org)     A   
                                 3    Rosella (subdariffa) petals (org)     A   
                                 4    Rosella (subdariffa) petals (org)     A   
...                                                                 ...   ...   
wc_purchases_deviations_bayoba_1 645           Baobab whole fruit (con)  None   
                                 646           Baobab whole fruit (con)  None   
                                 647           Baobab whole fruit (con)  None   
                                 648           Baobab whole fruit (con)  None   
                                 649           Baobab whole fruit (con)  None   

                                      amount_in_kg  price_per_kg  \
OGPur__OGMB                      0          15.260          0.85   
                                 1          74.555          0.85   
                                 2          61.855          0.85   
                                 3           6.955          0.85   
                                 4           7.885          0.85   
...                                            ...           ...   
wc_purchases_deviations_bayoba_1 645        64.000          0.10   
                                 646        10.000          0.10   
                                 647        32.000          0.10   
                                 648       134.000          0.10   
                                 649        52.000          0.10   

                                      total_payment_usd    buyer agr_year  \
OGPur__OGMB                      0                 13.0   Philip     1970   
                                 1                 63.0   Philip     1970   
                                 2                 53.0   Philip     1970   
                                 3                  6.0   Philip     1970   
                                 4                  7.0   Philip     1970   
...                                                 ...      ...      ...   
wc_purchases_deviations_bayoba_1 645     

In [153]:
concated_pur[['id']].squeeze().is_unique

False

In [154]:
table_name = 'all_purchase'

if engine!=None:
    try:
        concated_pur.to_sql(name=table_name, # Name of SQL table
                                con=engine, # Engine or connection
                                if_exists='replace', # Drop the table before inserting new values 
                                schema=schema, # Use schmea that was defined earlier
                                index=False, # Write DataFrame index as a column
                                chunksize=5000, # Specify the number of rows in each batch to be written at a time
                                method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The all_purchase table was imported successfully.
